## TensorFlow model saving

In [35]:
# Set the seed value for the notebook so the results are reproducible
from numpy.random import seed
seed(1)

In [36]:
import numpy as np
import pandas as pd

import warnings
warnings.simplefilter('ignore', FutureWarning)

## Data Pre-Processing

In [37]:
voice = pd.read_csv('data/voice.csv')

X = voice.drop("label", axis=1)
y = voice["label"]
print(X.shape, y.shape)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Step 1: Label-encode data set, label-encoder to transform "male", "Female" into code 0, 1
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding, to transform 0, 1 code into [1,0] and [0, 1]
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

(3168, 20) (3168,)


In [38]:
X

meanfreq        sd    median       Q25       Q75       IQR       skew  \
0     0.059781  0.064241  0.032027  0.015071  0.090193  0.075122  12.863462   
1     0.066009  0.067310  0.040229  0.019414  0.092666  0.073252  22.423285   
2     0.077316  0.083829  0.036718  0.008701  0.131908  0.123207  30.757155   
3     0.151228  0.072111  0.158011  0.096582  0.207955  0.111374   1.232831   
4     0.135120  0.079146  0.124656  0.078720  0.206045  0.127325   1.101174   
...        ...       ...       ...       ...       ...       ...        ...   
3163  0.131884  0.084734  0.153707  0.049285  0.201144  0.151859   1.762129   
3164  0.116221  0.089221  0.076758  0.042718  0.204911  0.162193   0.693730   
3165  0.142056  0.095798  0.183731  0.033424  0.224360  0.190936   1.876502   
3166  0.143659  0.090628  0.184976  0.043508  0.219943  0.176435   1.591065   
3167  0.165509  0.092884  0.183044  0.070072  0.250827  0.180756   1.705029   

             kurt    sp.ent       sfm      mode  centroid   meanfun    minfun  \
0      274.402906  0.893369  0.491918  0.000000  0.059781  0.084279  0.015702   
1      634.613855  0.892193  0.513724  0.000000  0.066009  0.107937  0.015826   
2     1024.927705  0.846389  0.478905  0.000000  0.077316  0.098706  0.015656   
3        4.177296  0.963322  0.727232  0.083878  0.151228  0.088965  0.017798   
4        4.333713  0.971955  0.783568  0.104261  0.135120  0.106398  0.016931   
...           ...       ...       ...       ...       ...       ...       ...   
3163     6.630383  0.962934  0.763182  0.200836  0.131884  0.182790  0.083770   
3164     2.503954  0.960716  0.709570  0.013683  0.116221  0.188980  0.034409   
3165     6.604509  0.946854  0.654196  0.008006  0.142056  0.209918  0.039506   
3166     5.388298  0.950436  0.675470  0.212202  0.143659  0.172375  0.034483   
3167     5.769115  0.938829  0.601529  0.267702  0.165509  0.185607  0.062257   

        maxfun   meandom    mindom    maxdom   dfrange   modindx  
0     0.275862  0.007812  0.007812  0.007812  0.000000  0.000000  
1     0.250000  0.009014  0.007812  0.054688  0.046875  0.052632  
2     0.271186  0.007990  0.007812  0.015625  0.007812  0.046512  
3     0.250000  0.201497  0.007812  0.562500  0.554688  0.247119  
4     0.266667  0.712812  0.007812  5.484375  5.476562  0.208274  
...        ...       ...       ...       ...       ...       ...  
3163  0.262295  0.832899  0.007812  4.210938  4.203125  0.161929  
3164  0.275862  0.909856  0.039062  3.679688  3.640625  0.277897  
3165  0.275862  0.494271  0.007812  2.937500  2.929688  0.194759  
3166  0.250000  0.791360  0.007812  3.593750  3.585938  0.311002  
3167  0.271186  0.227022  0.007812  0.554688  0.546875  0.350000  

[3168 rows x 20 columns]

In [39]:
y_test

2485    female
1611    female
1648    female
3027    female
2281    female
         ...  
2558    female
1153      male
2182    female
2217    female
2739    female
Name: label, Length: 792, dtype: object

In [40]:
# notice [1,0] is female, and [0,1] is male
y_test_categorical

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [41]:
y_train_categorical.shape

(2376, 2)

## Create a Deep Learning Model

In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=20)) #notice X.sape => (3168, 20)
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))    #notice y_train_categorical.shape => (2376, 2)

In [43]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=0
)

## Quantify our Trained Model

In [44]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

25/25 - 0s - loss: 0.0563 - accuracy: 0.9823
Normal Neural Network - Loss: 0.056322917342185974, Accuracy: 0.9823232293128967


## Make Predictions notice: label_encoder.inverse_transform(encoded_predictions)

In [45]:
#X_test_scaled[:5]

In [46]:
from numpy import argmax
#encoded_predictions = model.predict_classes(X_test_scaled[:5])
encoded_predictions = np.argmax(model.predict(X_test_scaled[:5]), axis=-1)
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [47]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: ['female' 'female' 'female' 'female' 'female']
Actual Labels: ['female', 'female', 'female', 'female', 'female']


## second predict way

In [56]:
# take a row 3 as an example, the real y is y[3]
Xtest = np.array(X.iloc[3, :])
Xtest = np.array([Xtest])
Xtest

array([[0.15122809, 0.07211059, 0.15801119, 0.09658173, 0.20795525,
        0.11137352, 1.23283128, 4.17729621, 0.96332246, 0.7272318 ,
        0.08387819, 0.15122809, 0.08896485, 0.01779755, 0.25      ,
        0.2014974 , 0.0078125 , 0.5625    , 0.5546875 , 0.24711908]])

In [57]:
Xtest_scaled = X_scaler.transform(Xtest)
ytest_predicted = model.predict(Xtest_scaled)
ytest_predicted

array([[2.2991155e-05, 9.9997699e-01]], dtype=float32)

In [58]:
from numpy import argmax
ypredicted = int(argmax(ytest_predicted))
ypredicted

1

In [59]:
#Predicted y
original_y = label_encoder.inverse_transform([ypredicted])
original_y

array(['male'], dtype=object)

In [60]:
#actual y
y[3]

'male'

## Saving a Trained Model

In [52]:
# Save the model
model.save("voice_model_trained.h5")

## Loading a Model 

In [53]:
# Load the model
from tensorflow.keras.models import load_model
voice_model = load_model("voice_model_trained.h5")

## Evaluating the loaded model

In [54]:
voice_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [55]:
model_loss, model_accuracy = voice_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

25/25 - 0s - loss: 0.0563 - accuracy: 0.9823
Normal Neural Network - Loss: 0.056322917342185974, Accuracy: 0.9823232293128967
